# Assignment 2: Entity Resolution (Part 2)


## Objective

In Assignment 2 (Part 2), you will learn how to use Active Learning to address the entity resolution problem. After completing this assignment, you should be able to answer the following questions:

1. Why Active Learning?
2. How to implement uncertain sampling, a popular query strategy for Active Learning?
3. How to solve an ER problem using Active Learning?


## Active Learning

[Active learning](http://tiny.cc/al-wiki) is a certain type of ML algorithms that can train a high-quality ML model with small data-labeling cost. Its basic idea is quite easy to understand. Consider a typical supervised ML problem, which requires a (relatively) large training dataset. In the training dataset, there may be only a small number of data points that are beneficial to the trained ML model. In other words, labeling a small number of data points is enough to train a high-quality ML model. The goal of active learning is to help us to identify those data points. 


In this assignment, we will develop an Active Learning approach for Entity Resolution. The following figure shows the architecture of an entity resolution solution. It consists of four major steps. **The source code for Steps 1, 2, 4 are provided below. Your job is to implement Step 3.**  


![title](img/arch.png)

### Step 1. Read Data

The datasets are under `A2-data/part2-active-learning`. Suppose we get a restaurant dataset `restaurant.csv`. The data has many duplicate restaurants.  For example, the first two rows shown below are duplicated (i.e., refer to the same real-world entity). You can check out all duplicate (matching) record pairs from `true_matches.json`. 

In [1]:
import pandas as pd
# you may need to modify the file path
df = pd.read_csv('A2-data/part2-active-learning/restaurant.csv')
data = df.values.tolist()
print("(#Rows, #Cols) :", df.shape)
df.head(5)

(#Rows, #Cols) : (858, 5)


,id,name,address,city,type
0,1,arnie morton's of chicago,435 s. la cienega blv.,los angeles,american
1,2,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,steakhouses
2,3,art's delicatessen,12224 ventura blvd.,studio city,american
3,4,art's deli,12224 ventura blvd.,studio city,delis
4,5,hotel bel-air,701 stone canyon rd.,bel air,californian


### Step 2. Similar Pairs

We first use a similarity-join algorithm to generate similar pairs. 

Below is the code. After running the code, we get 678 similar pairs ordered by their similarity decreasingly.

In [2]:
import sys

# Add a folder to Python's "search path"
# This allows Python to find and import files (modules) inside that folder.
# Here, we want to import a2_utils.py from this folder:
sys.path.insert(1, 'A2-data/part2-active-learning/')

# Import everything (*) from a2_utils.py
# a2_utils.py contains helper functions, including:
# - simjoin(): makes "similar pairs"
# - featurize(): makes feature vectors (used later)
# - crowdsourcing(): simulates human labeling (used later)
from a2_utils import *

# Convert the pandas DataFrame (df) into a Python list of rows
# Each row is one restaurant record:
# [id, name, address, city, type]
data = df.values.tolist()

# Run similarity join (blocking step)
# simjoin(data) compares records and returns only the pairs that look similar
# This reduces a HUGE number of all possible pairs to a much smaller candidate set.
simpairs = simjoin(data)

# Print the total number of possible pairs if we compare every record with every other record
# n*(n-1)/2 is the number of unique pairs (combinations)
print("Num of Pairs: ", len(data)*(len(data)-1)/2)

# Print how many "similar pairs" we kept after simjoin
print("Num of Similar Pairs: ", len(simpairs))

# simpairs is sorted by similarity from high to low
# So simpairs[0] is the most similar pair
print("The Most Similar Pair: ", simpairs[0])

Num of Pairs:  367653.0
Num of Similar Pairs:  678
The Most Similar Pair:  ([170, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern/soul'], [169, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern'])


We can see that `simjoin` helps us remove the number of pairs from 367653 to 678. But, there are still many non-matching pairs in `simpairs` (see below). 

In [3]:
print(simpairs[-1])

([764, "buzio's in the rio", '3700 w. flamingo rd.', 'las vegas', 'seafood'], [542, 'carnival world', '3700 w. flamingo rd.', 'las vegas', 'buffets'])


Next, we will use active learning to train a classifier, and then use the classifier to classify each pair in `simpairs` as either "matching" or "nonmatching". 

### Step 3. Active Learning

Given a set of similar pairs, what you need to do next is to iteratively train a classifier to decide which pairs are truly matching. We are going to use [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) as our classifier. 

#### Initialization

At the beginning, all the pairs are unlabeled. To initialize a model, we first pick up ten pairs and then label each pair using  the `crowdsourcing()` function. You can assume that `crowdsourcing()` will ask a crowd worker (e.g., on Amazon Mechanical Turk) to label a pair. 


`crowdsourcing(pair)` is a function that simulates the use of crowdsourcing to label a pair
  
  - **Input:**	pair – A pair of records 

  - **Output:**	Boolean –  *True*: The pair of records are matching; *False*: The pair of records are NOT matching;

Please use the following code to do the initialization. 

In [11]:
# Import the crowdsourcing function
# This function simulates asking a human to label a pair
from a2_utils import crowdsourcing

# Choose the 5 most similar pairs and the 5 least similar pairs
# simpairs is already sorted by similarity
init_pairs = simpairs[:5] + simpairs[-5:]

# Create empty lists to store labeled pairs
matches = []      # pairs that are truly the same entity
nonmatches = []   # pairs that are NOT the same entity

# Loop through the 10 initial pairs
for pair in init_pairs:

    # Ask the "human" (simulated) if this pair is a match
    is_match = crowdsourcing(pair)

    # If the answer is True, the records are the same
    if is_match == True:
        matches.append(pair)

    # Otherwise, they are different
    else:
        nonmatches.append(pair)

# Print how many initial labels we got
print("Number of matches: ", len(matches))
print("Number of nonmatches: ", len(nonmatches))

Are they matching?
[170, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern/soul']
[169, "mary mac's tea room", '224 ponce de leon ave.', 'atlanta', 'southern']
Answer: Yes
Are they matching?
[88, 'manhattan ocean club', '57 w. 58th st.', 'new york city', 'seafood']
[87, 'manhattan ocean club', '57 w. 58th st.', 'new york', 'seafood']
Answer: Yes
Are they matching?
[112, 'san domenico', '240 central park s.', 'new york city', 'italian']
[111, 'san domenico', '240 central park s', 'new york', 'italian']
Answer: Yes
Are they matching?
[197, 'fleur de lys', '777 sutter st.', 'san francisco', 'french (new)']
[196, 'fleur de lys', '777 sutter st.', 'san francisco', 'french']
Answer: Yes
Are they matching?
[8, 'cafe bizou', '14016 ventura blvd.', 'sherman oaks', 'french bistro']
[7, 'cafe bizou', '14016 ventura blvd.', 'sherman oaks', 'french']
Answer: Yes
Are they matching?
[709, 'arcadia', '21 e. 62nd st.', 'new york city', 'american (new)']
[66, 'four seasons', '99 e. 5

Here is the only code you need to write in this assignment.


In [8]:
# We will use two helper functions from a2_utils.py:
# 1) featurize(pair): converts one pair into a numeric feature vector (numbers).
# 2) crowdsourcing(pair): returns True/False as the label (match or not match).
from a2_utils import featurize, crowdsourcing

# labeled_pairs = pairs that already have labels (True/False)
# matches and nonmatches were created in the Initialization step (5 + 5 = 10)
labeled_pairs = matches + nonmatches

# unlabeled_pairs = pairs that do NOT have labels yet
# We start with all simpairs and remove those that are already labeled
unlabeled_pairs = [p for p in simpairs if p not in labeled_pairs]

# Number of active learning iterations
# Each iteration: train -> find most uncertain pair -> ask crowdsourcing -> update sets
iter_num = 5

# ------------------------------------------------------------
# Step 3: Active Learning with Uncertainty Sampling (YOUR CODE)
# ------------------------------------------------------------

# Logistic Regression is a simple binary classifier (0/1 classification).
# Here: 1 = match, 0 = nonmatch
from sklearn.linear_model import LogisticRegression

# numpy is used for arrays and math operations
import numpy as np

# random is used to break ties randomly (if multiple pairs are equally uncertain)
import random

# Fix the random seed so tie-breaking is reproducible (same result every run)
random.seed(42)

# ------------------------------------------------------------
# Helper function: build training data (X, y) from labeled pairs
# ------------------------------------------------------------
def make_xy(pairs, matches):
    """
    pairs: a list of labeled pairs (each element is a pair of two records)
    matches: a list containing pairs that are labeled as MATCH

    return:
      X: feature matrix (2D array). Each row is a feature vector for one pair.
      y: label vector (1D array). 1 = match, 0 = nonmatch.
    """

    # Convert every pair into a numeric feature vector using featurize()
    # Example of one feature vector: [name_similarity, address_similarity, ...]
    X = np.array([featurize(p) for p in pairs])

    # Create labels:
    # If a pair is in matches_set, label is 1 (match), else 0 (nonmatch)
    y = np.array([1 if p in matches else 0 for p in pairs])

    return X, y

# # ------------------------------------------------------------
# # Use sets for fast membership check (p in matches_set is fast)
# # ------------------------------------------------------------
# matches_set = set(matches)          # initial match pairs (5)
# nonmatches_set = set(nonmatches)    # initial nonmatch pairs (5) (not strictly needed, but kept for clarity)

# ------------------------------------------------------------
# Active Learning Loop
# ------------------------------------------------------------
for it in range(iter_num):

    # 1) Build the labeled training set for this iteration
    labeled_pairs = matches + nonmatches

    # Convert labeled pairs into training data (X_train, y_train)
    X_train, y_train = make_xy(labeled_pairs, matches)

    # 2) Train a Logistic Regression model
    # - max_iter=1000: allow enough iterations for optimization to converge
    # - class_weight="balanced": helps if classes are imbalanced (few matches, many nonmatches)
    model = LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42)
    model.fit(X_train, y_train)

    # 3) Apply the model to all unlabeled pairs and get probability for "match"
    # Convert unlabeled pairs into feature vectors
    X_unlab = np.array([featurize(p) for p in unlabeled_pairs])

    # predict_proba gives probabilities for each class:
    # probs[:, 0] = probability of class 0 (nonmatch)
    # probs[:, 1] = probability of class 1 (match)
    probs = model.predict_proba(X_unlab)[:, 1]  # take the match probability only

    # 4) Uncertainty Sampling:
    # A pair is MOST uncertain when the model is closest to 0.5 probability.
    # uncertainty = |p - 0.5|
    uncertainty = np.abs(probs - 0.5)

    # Find the minimum uncertainty value
    min_u = np.min(uncertainty)

    # 5) If there is a tie (multiple pairs have the same min uncertainty),
    # choose one randomly.
    candidate_idxs = np.where(uncertainty == min_u)[0].tolist()
    chosen_idx = random.choice(candidate_idxs)

    # Get the chosen pair and its probability
    chosen_pair = unlabeled_pairs[chosen_idx]
    chosen_prob = probs[chosen_idx]

    # 6) Ask "human" to label the chosen pair (simulated by crowdsourcing())
    # is_match is True if match, False if nonmatch
    is_match = crowdsourcing(chosen_pair)

    # 7) Update labeled sets:
    # Move chosen_pair from unlabeled to labeled
    if is_match:
        # Add to matches list and set
        matches.append(chosen_pair)
    else:
        # Add to nonmatches list and set
        nonmatches.append(chosen_pair)

    # Remove chosen pair from unlabeled list
    unlabeled_pairs.pop(chosen_idx)

    # Print progress (for debugging / understanding)
    print(
        f"[Iter {it+1}/{iter_num}] p(match)={chosen_prob:.4f} -> "
        f"{'MATCH' if is_match else 'NONMATCH'} | "
        f"labeled={len(matches)+len(nonmatches)}, unlabeled={len(unlabeled_pairs)}"
    )

# ------------------------------------------------------------
# Final step: train the final model one more time on all labeled data
# and keep it in variable 'model' (required output)
# ------------------------------------------------------------
labeled_pairs = matches + nonmatches
X_train, y_train = make_xy(labeled_pairs, matches)

model = LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42)
model.fit(X_train, y_train)

Are they matching?
[4, "art's deli", '12224 ventura blvd.', 'studio city', 'delis']
[3, "art's delicatessen", '12224 ventura blvd.', 'studio city', 'american']
Answer: Yes
[Iter 1/5] p(match)=0.5091 -> MATCH | labeled=16, unlabeled=662
Are they matching?
[217, 'ritz-carlton dining room (san francisco)', '600 stockton st.', 'san francisco', 'french (new)']
[216, 'ritz-carlton restaurant and dining room', '600 stockton st.', 'san francisco', 'american']
Answer: Yes
[Iter 2/5] p(match)=0.5141 -> MATCH | labeled=17, unlabeled=661
Are they matching?
[84, 'lespinasse (new york city)', '2 e. 55th st.', 'new york city', 'asian']
[83, 'lespinasse', '2 e. 55th st.', 'new york', 'american']
Answer: Yes
[Iter 3/5] p(match)=0.4973 -> MATCH | labeled=18, unlabeled=660
Are they matching?
[545, 'empress court', '3570 las vegas blvd. s', 'las vegas', 'asian']
[137, 'palace court', '3570 las vegas blvd. s', 'las vegas', 'continental']
Answer: No
[Iter 4/5] p(match)=0.4970 -> NONMATCH | labeled=19, unlab

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


**[Algorithm Description].**   Active learning has many [query strategies](http://tiny.cc/al-wiki-qs) to decide which data point should be labeled. You need to implement uncertain sampling. The algorithm trains an initial model on `labeled_pairs`. Then, it iteratively trains a model. At each iteration, it first applies the model to `unlabeled_pairs`, and makes a prediction on each unlabeled pair along with a probability, where the probability indicates the confidence of the prediction. After that, it selects the most uncertain pair (If there is still a tie, break it randomly),  and call the `crowdsourcing()` function to label the pair. After the pair is labeled, it updates `labeled_pairs` and `unlabeled_pairs`, and then retrain the model on `labeled_pairs`.

**[Input].** 
- `labeled_pairs`: 10 labeled pairs (by default)
- `unlabeled_pairs`: 668 unlabeled pairs (by default)
- `iter_num`: 5 (by default)

**[Output].** 
- `model`: A logistic regression model built with scikit-learn


### Step 4. Model Evaluation

After training an model, you can use the following code to evalute it.

In [10]:
import json
# We will use the evaluate() function to calculate precision, recall, and F1 score
from a2_utils import evaluate

# Convert every similar pair into a numeric feature vector
# simpairs = all candidate record pairs (678 pairs)
# featurize(sp) turns one pair into a list of numbers
sp_features = np.array([featurize(sp) for sp in simpairs])

# Use the trained model to predict labels for every pair
# model.predict() returns:
#   1 = match
#   0 = nonmatch
label = model.predict(sp_features)

# Zip each pair together with its predicted label
# Example: (pair1, 1), (pair2, 0), ...
pair_label = zip(simpairs, label)

# Create an empty list to store only predicted matches
identified_matches = []

# Go through every (pair, label)
for pair, label in pair_label:

    # If the model says this pair is a match
    if label == 1:

        # Save this pair as a predicted match
        identified_matches.append(pair)

# Compare our predicted matches with the true answers
# evaluate() reads true_matches.json internally
# and calculates precision, recall, and F1 score
precision, recall, fscore = evaluate(identified_matches)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("Fscore:", fscore)

Precision: 0.9504950495049505
Recall: 0.9056603773584906
Fscore: 0.927536231884058


## Submission

Complete the code in A2-2.ipynb, and submit it to Canvas.